# model_selection

用於模型選擇和評估。這個子模塊提供了一系列函數和工具，幫助用戶進行模型的選擇、優化和評估，以提高機器學習模型的性能和泛化能力。

分割資料 和 交叉驗證 時常會 import 此模塊

## 常用的函數

1. train_test_split：用於將數據集分割為訓練集和測試集。

    * train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)

    * arrays：要分割的數據集和標籤（特徵矩陣和目標向量）。
    
    * test_size（float 或 int 或 None）：測試集大小，可以是浮點數（表示測試集的比例）或整數（表示測試集的樣本數）。
    
    * random_state（int 或 RandomState 或 None）：隨機種子，用於控制隨機分割的結果。
    
    * shuffle（bool）：是否在分割之前對數據進行隨機打亂。
    
    * stratify（array-like 或 None）：用於分層抽樣，確保訓練集和測試集中的類別比例與原始數據集一致（僅在分類問題中使用）。
---
2. StratifiedShuffleSplit：分層隨機抽樣分割器，用於將數據集進行分層隨機抽樣，生成訓練集和測試集。

    * StratifiedShuffleSplit(n_splits, test_size, train_size, random_state)

    * n_splits（int）：分割數量，即要劃分的訓練集和測試集對的數量。
---
3. cross_val_score：交叉驗證函數，用於對模型進行交叉驗證，評估模型的性能。它將數據集分成 k 折，進行 k 次訓練和測試，得到 k 個評估指標，用於衡量模型的性能。

    * cross_val_score(estimator, X, y, cv, scoring, n_jobs, ...)

    * X（array-like）：特徵矩陣。

    * y（array-like）：目標向量。

    * cv（int 或 交叉驗證生成器）：交叉驗證策略，可以是整數（表示 K 折交叉驗證）或交叉驗證生成器。

    * scoring（string 或 callable）：評估指標，用於計算得分。cross_val_score 會更具指定的模型選擇評分方式, 回歸是使用r2 score, 分類問題是使用 Classification Accuracy

    * n_jobs（int）：並行運算的作業數。如果設置為 -1，則使用所有可用的 CPU。

    * 其他參數：例如 groups（用於群組交叉驗證）等。
---
3. KFold 和 StratifiedKFold：交叉驗證的分割器類，用於將數據集分成 k 折進行交叉驗證。KFold 是普通的 k 折交叉驗證，StratifiedKFold 是分層抽樣的 k 折交叉驗證，確保每個折中的類別比例與原始數據集保持一致。

    * KFold(n_splits, shuffle, random_state)

    * n_splits（int）：折數 K。

    * shuffle（bool）：是否在分割之前對數據進行隨機打亂。默認為 False。
---
4. GridSearchCV：網格搜索交叉驗證，用於對模型進行超參數的網格搜索和交叉驗證，尋找最佳的超參數組合，以提高模型性能。

    * GridSearchCV(estimator, param_grid, cv, scoring, n_jobs, ...)

    * estimator：待調參的模型。

    * param_grid：參數網格，需要嘗試的參數組合。

    * cv：交叉驗證策略，可以是整數（表示 K 折交叉驗證）或交叉驗證生成器。

    * scoring：評估指標，用於選擇最佳參數。
---
5. RandomizedSearchCV：隨機搜索交叉驗證，與 GridSearchCV 類似，但是它是對超參數進行隨機搜索，適用於超參數空間很大的情況。

    * RandomizedSearchCV(estimator, param_distributions, n_iter, cv, scoring, n_jobs, ...)

    * param_distributions：一個字典，包含了待搜索的超參數和其對應的取值範圍。字典的 key 是超參數的名稱，value 是一個取值範圍，可以是單一數值、數值範圍、列表等。

    * n_iter：進行隨機搜索的迭代次數。隨機搜索會在 param_distributions 定義的超參數空間中隨機選取 n_iter 組參數組合進行評估。
---
6. learning_curve：學習曲線函數，用於繪製模型的學習曲線，顯示模型在訓練集和測試集上的性能隨著數據量的變化。

    * learning_curve(estimator, X, y, train_sizes, cv, scoring, n_jobs, ...)
---
7. validation_curve：驗證曲線函數，用於繪製模型的驗證曲線，顯示模型在不同超參數下的性能變化，幫助選擇最佳的超參數。

    * validation_curve(estimator, X, y, param_name, param_range, cv, scoring, n_jobs, ...)

    * param_name（string）：參數名稱。

    * param_range（array-like）：參數值範圍。

### 分割
[參考資料](https://ithelp.ithome.com.tw/articles/10290884)

train_test_split 和 StratifiedShuffleSplit 的差別在於 :
* train_test_split : 一次只能分割整組資料
* StratifiedShuffleSplit : 可以依照給定的組別, 每組個別去分割. 這又能有效讓資料跟接近原始資料.

兩種分割皆可以依照 label 的比例去分割(分割後 label 的比例接近).


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

housing = pd.read_csv("https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv")

In [8]:
# 新增 "income_cat" 的 col, 其值為 1 至 5, 分別對應 median_income 那個 col 上值介於0~1.5、1.5~3.0,...,6.0~inf 的值.
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

#每個分組都取20%放入測試集
Split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)  #設定分割方法, n_splits 為交叉驗證的折(fold)數.

for train_index, test_index in Split.split(housing, housing["income_cat"]): #雖然寫成for, 不過只會進行一次運算, 因為前面 n_splits 為 1
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [9]:
# 比較比例
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size = 0.2, random_state=42) # 使用 train_test_split 分割的結果

compare_props = pd.DataFrame({
                "Overall": income_cat_proportions(housing), # 原始比例
                "Stratified": income_cat_proportions(strat_test_set), # 使用 StratifiedShuffleSplit 分割後的比例
                "Random": income_cat_proportions(test_set) # 使用 train_test_split 分割後的比例
                }).sort_index() # 依照index 排序
compare_props

,Overall,Stratified,Random
1,0.039826,0.039971,0.040213
2,0.318847,0.318798,0.324370
3,0.350581,0.350533,0.358527
4,0.176308,0.176357,0.167393
5,0.114438,0.114341,0.109496


### 交叉驗證

交叉驗證在 sklearn.model_selection 有以下幾個函數可以使用, 函數若使計算 score, 回歸的默認方式是 R^2, 分類問題是使用 Classification Accuracy.

* cross_val_score(estimator, X, y, cv, scoring, n_jobs, ...) :

  會直接算出 estimator 的 cv score.

* KFold(n_splits, shuffle, random_state)

  得到的會是 train 和 test 的 index, 必須自己去 fit model 求 cv score.

* GridSearchCV(estimator, param_grid, cv, scoring, n_jobs, ...)

  會計算各個超參數的 cv score, 

* RandomizedSearchCV(estimator, param_distributions, n_iter, cv, scoring, n_jobs, ...)

---

#### Score 的計算方式有以下幾個, 以 GridSearchCV 為例 :

* R^2 :
```python
    GridSearchCV(estimator, param_grid, scoring='r2', cv=5)
```
* 平均絕對誤差（Mean Absolute Error，MAE） :
```python
    from sklearn.metrics import make_scorer, mean_absolute_error
    
    scoring = make_scorer(mean_absolute_error)
    grid_search = GridSearchCV(estimator, param_grid, scoring=scoring, cv=5)
```
* 均方誤差（Mean Squared Error，MSE） :
```python
    from sklearn.metrics import make_scorer, mean_squared_error

    scoring = make_scorer(mean_squared_error)
    grid_search = GridSearchCV(estimator, param_grid, scoring=scoring, cv=5)
```
* 對數似然（Log Likelihood） :
```python
    from sklearn.metrics import make_scorer, log_loss

    scoring = make_scorer(log_loss)
    grid_search = GridSearchCV(estimator, param_grid, scoring=scoring, cv=5)
```

---

#### make_scorer 函數的作用是將一個原始的評分函數轉換成 Scikit-learn 中的評分器（scorer）

使用範例
```python
from sklearn.metrics import make_scorer, mean_squared_error

# 定義自定義的評分函數，例如均方誤差（MSE）
def custom_mse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

# 將自定義的評分函數轉換為評分器
scorer = make_scorer(custom_mse)

# 現在 scorer 就是一個符合 Scikit-learn 評分器接口的對象，可以用於交叉驗證或網格搜索
```

#### python

分析 boston data.
1. 使用分割(train_test_split) 將資料分成 train 和 test
2. model 使用 ridge 或 lasso, 超參數(alpha) 從 np.logspace(-3, 3, 13) 這 13 個值決定
3. 交叉驗證的評分(score) 方式為 R^2 (默認方式)

使用 GridSearchCV :

In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
import warnings

# 忽略警告訊息
warnings.filterwarnings("ignore")

# 載入波士頓房價數據集
boston = load_boston()
X, y = boston.data, boston.target

# 將數據集分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 10) # 分割資料

# 建立 Ridge 和 Lasso 回歸模型
ridge = Ridge()
lasso = Lasso()

# 定義超參數網格，這裡我們只調整正則化參數 alpha
param_grid = {'alpha': np.logspace(-3, 3, 13)}

# 使用 5-fold 交叉驗證進行網格搜索
grid_search_ridge = GridSearchCV(ridge, param_grid, scoring='r2', cv=5)
grid_search_lasso = GridSearchCV(lasso, param_grid, scoring='r2', cv=5)

# 訓練 Ridge 和 Lasso 回歸模型
grid_search_ridge.fit(X_train, y_train)
grid_search_lasso.fit(X_train, y_train)

# 打印最佳超參數組合和交叉驗證得分
print("Ridge 最佳超參數組合:", grid_search_ridge.best_params_)
print("Ridge 交叉驗證:", cross_val_score(grid_search_ridge.best_estimator_, X_train, y_train, cv=5).mean())

print("Lasso 最佳超參數組合:", grid_search_lasso.best_params_)
print("Lasso 交叉驗證:", cross_val_score(grid_search_lasso.best_estimator_, X_train, y_train, cv=5).mean())

print('-'*20)

# 使用超參數分析資料
grid_search_ridge.best_estimator_.fit(X_train, y_train)
y_ridge_pred = grid_search_ridge.best_estimator_.predict(X_test)

grid_search_lasso.best_estimator_.fit(X_train, y_train)
y_lasso_pred = grid_search_lasso.best_estimator_.predict(X_test)

# 計算均方誤差
ridge_mse = np.mean((y_test - y_ridge_pred) ** 2)
lasso_mse = np.mean((y_test - y_lasso_pred) ** 2)

print("Ridge 均方誤差:", ridge_mse)
print("Lasso 均方誤差:", lasso_mse)

Ridge 最佳超參數組合: {'alpha': 0.31622776601683794}
Ridge 交叉驗證: 0.7189676987418181
Lasso 最佳超參數組合: {'alpha': 0.0031622776601683794}
Lasso 交叉驗證: 0.7185312058831054
--------------------
Ridge 均方誤差: 34.620632912364236
Lasso 均方誤差: 34.509845273708855



---
#### 使用 cross_val_score

In [11]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, train_test_split
import warnings

# 忽略警告訊息
warnings.filterwarnings("ignore")

# 載入波士頓房價數據集
boston = load_boston()
X, y = boston.data, boston.target

# 將數據集分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 10) # 分割資料

# 建立 Ridge 和 Lasso 回歸模型
ridge = Ridge()
lasso = Lasso()

# 定義超參數收尋列表
param_grid = np.logspace(-3, 3, 13)
ridge_score_mean = []
lasso_score_mean = []
for Param in param_grid:
    
    # 建立 Ridge 和 Lasso 回歸模型
    ridge_model = Ridge(alpha = Param)
    lasso_model = Lasso(alpha = Param)

    ridge_score_mean.append(cross_val_score(ridge_model, X_train, y_train, cv = 5).mean())
    lasso_score_mean.append(cross_val_score(lasso_model, X_train, y_train, cv = 5).mean())

# R^2 最大值之超參數位置
ridge_min_index = ridge_score_mean.index(max(ridge_score_mean))
lasso_min_index = lasso_score_mean.index(max(lasso_score_mean))

# 最佳超參數
ridge_best_params = param_grid[ridge_min_index]
lasso_best_params = param_grid[lasso_min_index]

# 使用最佳的 cv socre 得到的超參數, 建立模型
ridge_model = Ridge(alpha = ridge_best_params)
lasso_model = Lasso(alpha = lasso_best_params)

print("Ridge 最佳超參數組合:", ridge_best_params)
print("Ridge 交叉驗證:", cross_val_score(ridge_model, X_train, y_train, cv=5).mean())

print("Lasso 最佳超參數組合:", ridge_best_params)
print("Lasso 交叉驗證:", cross_val_score(lasso_model, X_train, y_train, cv=5).mean())

print('-'*20)

# 模型使用最佳的超參數去fit data
ridge_model.fit(X_train, y_train)
y_ridge_pred = ridge_model.predict(X_test)

lasso_model.fit(X_train, y_train)
y_lasso_pred = lasso_model.predict(X_test)

# 計算均方誤差
ridge_mse = np.mean((y_test - y_ridge_pred) ** 2)
lasso_mse = np.mean((y_test - y_lasso_pred) ** 2)

print("Ridge 均方誤差:", ridge_mse)
print("Lasso 均方誤差:", lasso_mse)


Ridge 最佳超參數組合: 0.31622776601683794
Ridge 交叉驗證: 0.7189676987418181
Lasso 最佳超參數組合: 0.31622776601683794
Lasso 交叉驗證: 0.7185312058831054
--------------------
Ridge 均方誤差: 34.620632912364236
Lasso 均方誤差: 34.509845273708855



---
使用 KFold

In [12]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score, train_test_split, KFold
import warnings

# 忽略警告訊息
warnings.filterwarnings("ignore")

# 載入波士頓房價數據集
boston = load_boston()
X, y = boston.data, boston.target

# 將數據集分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 10) # 分割資料

# 建立 Ridge 和 Lasso 回歸模型
ridge = Ridge()
lasso = Lasso()

# 定義超參數收尋列表
param_grid = np.logspace(-3, 3, 13)
ridge_score_mean = []
lasso_score_mean = []

kf = KFold(n_splits = 5, shuffle = True, random_state = 20)
for Param in param_grid:
    # 建立模型
    ridge_model = Ridge(alpha = Param)
    lasso_model = Lasso(alpha = Param)
    ridge_score_list = [] #放置 score
    lasso_score_list = []

    for train_index, test_index in kf.split(X_train):
        # 分割
        CV_X_train, CV_X_test = X_train[train_index], X_train[test_index]
        CV_y_train, CV_y_test = y_train[train_index], y_train[test_index]

        # fit model
        ridge_model.fit(CV_X_train, CV_y_train) 
        lasso_model.fit(CV_X_train, CV_y_train)

        # 計算 cv scores
        ridge_score_list.append(ridge_model.score(CV_X_test, CV_y_test))
        lasso_score_list.append(lasso_model.score(CV_X_test, CV_y_test))
    
    # 計算平均的 cv score
    ridge_score_mean.append(np.mean(ridge_score_list))
    lasso_score_mean.append(np.mean(lasso_score_list))

# R^2 最大值之超參數位置
ridge_min_index = ridge_score_mean.index(max(ridge_score_mean))
lasso_min_index = lasso_score_mean.index(max(lasso_score_mean))

# 最佳超參數
ridge_best_params = param_grid[ridge_min_index]
lasso_best_params = param_grid[lasso_min_index]

# 使用最佳的 cv socre 得到的超參數, 建立模型
ridge_model = Ridge(alpha = ridge_best_params)
lasso_model = Lasso(alpha = lasso_best_params)

print("Ridge 最佳超參數組合:", ridge_best_params)
print("Ridge 交叉驗證:", cross_val_score(ridge_model, X_train, y_train, cv=5).mean())

print("Lasso 最佳超參數組合:", ridge_best_params)
print("Lasso 交叉驗證:", cross_val_score(lasso_model, X_train, y_train, cv=5).mean())

print('-'*20)

# 模型使用最佳的超參數去fit data
ridge_model.fit(X_train, y_train)
y_ridge_pred = ridge_model.predict(X_test)

lasso_model.fit(X_train, y_train)
y_lasso_pred = lasso_model.predict(X_test)

# 計算均方誤差
ridge_mse = np.mean((y_test - y_ridge_pred) ** 2)
lasso_mse = np.mean((y_test - y_lasso_pred) ** 2)

print("Ridge 均方誤差:", ridge_mse)
print("Lasso 均方誤差:", lasso_mse)

Ridge 最佳超參數組合: 0.31622776601683794
Ridge 交叉驗證: 0.7189676987418181
Lasso 最佳超參數組合: 0.31622776601683794
Lasso 交叉驗證: 0.7185312058831054
--------------------
Ridge 均方誤差: 34.620632912364236
Lasso 均方誤差: 34.509845273708855
